In [ ]:
import bpwave
import bpwave.visu
import cam_bpw_sim as bps
import datetime as dt
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import pathlib as pl
import pandas as pd

sns.set_style("whitegrid")

In [ ]:
par_meas_file = ""
par_results_folder = ""

In [ ]:
if not par_meas_file or not par_results_folder:
    raise ValueError
meas_file = pl.Path(par_meas_file)
results_folder = pl.Path(par_results_folder)
results_folder.mkdir(exist_ok=True)

In [ ]:
mpl.rcParams["figure.figsize"] = (15, 5)

In [ ]:
dt.datetime.now().isoformat(), bpwave.__version__, bps.__version__

## Loading data

The input is the selected measurement file that can be generated with `cam_bpw_sim.app.convert_measurement_log`.
This file includes the measured time series and the configuration of the simulator environment as well.

In [ ]:
app_config = bps.app.read_config()

In [ ]:
with h5py.File(meas_file) as f:
    full_measured = bpwave.Signal.from_hdf(f["measured"])
    cam_signal_nom = bpwave.Signal.from_hdf(f["nominal"])
    cam_params = bps.cam.CamParams.from_hdf(f["cam_params"])
    cam_inst_params = bps.cam.CamInstance.from_hdf(f["cam_inst"])
    sim_params = bps.meas.SimulatorParams.from_hdf(f["simulator"])
    meas_params = bps.meas.MeasParams.from_hdf(f["meas_params"])

In [ ]:
cam_signal_nom.plot(legend="outside")

In [ ]:
full_measured.plot()

In [ ]:
bps.val.plot_observed_values(full_measured, figsize=(15, 5))

In [ ]:
ccycles_per_fcr = cam_signal_nom.onsets.size - 1
inverted = cam_params.invert
fs = full_measured.fs
meas_desc = f"{cam_params.name} {cam_inst_params.material} A={cam_params.amplitude}mm U={meas_params.u}V"

## Data preparation

In [ ]:
if inverted:
    full_measured = full_measured.copy(y=full_measured.y.max() - full_measured.y)

In protocols P3, P4 and P5, the motor is started with a cam position that corresponds to the saved nominal signal, so we don't need to synchronize.

In [ ]:
noise, measured = bps.val.split_combined_measurement(full_measured)

Characteristic point detection needs lowpass-filtered signal.

In [ ]:
point_detector = bps.signal.ScipyFindPeaks()
measured.chpoints = point_detector(bps.signal.denoise(measured)).chpoints

In [ ]:
bps.val.plot_combined_measurement(full_measured, noise, measured)

In [ ]:
plt.magnitude_spectrum(measured.y, Fs=measured.fs, scale="dB")

In [ ]:
t_wheel_turn = bps.val.t_cam_rotation(measured, ccycles_per_fcr)
t_wheel_turn

In [ ]:
amplitudes = np.array([cc.y.max() - cc.y.min() for cc in measured.iter_ccycles()])

In [ ]:
preproc = bps.val.preproc_for_validation(
    measured, cam_signal_nom, n_ccycles=ccycles_per_fcr, t_tol=0.1
)

In [ ]:
nominal_ccycles = [cc.shift_t() for cc in preproc.nominal_matched.ccycles[:]]

In [ ]:
bps.val.plot_signal_slices(preproc.measured_long_bw_corr, figsize=(15, 6))

In [ ]:
bps.val.plot_stacked_slices(
    preproc.measured_long_bw_corr,
    "fcr",
    overlay=preproc.nominal_matched,
    desc="Aligned full cam rotations without longterm baseline wander",
    onsets=False,
)

In [ ]:
bps.val.plot_signal_slices(preproc.measured_bw_corr, figsize=(15, 6))

In [ ]:
bps.val.plot_stacked_slices(
    preproc.measured_bw_corr,
    "fcr",
    overlay=preproc.nominal_matched,
    desc="Aligned full cam rotations without baseline wander",
    onsets=False,
)

In [ ]:
chpoints = bps.val.build_chpoints_table(
    preproc.measured_long_bw_corr, include_end_onset=True
)

### Onset statistics

In [ ]:
sns.relplot(chpoints, x="fcr", y="onset", hue="ccycle", marker="X")

Onset uncertainty of the measured signal (in terms of # of data points)

In [ ]:
chpoints[chpoints["ccycle"] < ccycles_per_fcr].drop(columns="fcr").groupby(
    ["ccycle"]
).agg(["mean", "std"]).T

## Cross comparison of full cam rotations

### Waveforms for cross comparison

In [ ]:
fcrs_long_bw_corr = bps.val.build_fcr_array(preproc.measured_long_bw_corr, "fcr")
fcrs_bw_corr = bps.val.build_fcr_array(preproc.measured_bw_corr, "fcr")

### Precision: RMSE (longterm BW removed)

In [ ]:
rmse_rep_fcrs_long_bw_corr = bps.val.cross_compare(
    fcrs_long_bw_corr,
    fcrs_long_bw_corr,
    bps.val.rmse_s,
)

In [ ]:
rmse_rep_fcrs_long_bw_corr.heatmaps()

In [ ]:
rmse_rep_fcrs_long_bw_corr.boxplot(figsize=(10, 2))

In [ ]:
rmse_rep_fcrs_long_bw_corr.examples(meas_desc, "RMSE (longterm BW removed)", good="min")

In [ ]:
pearson_rep_fcrs_long_bw_corr = bps.val.cross_compare(
    fcrs_long_bw_corr,
    fcrs_long_bw_corr,
    bps.val.pearson_s,
)

In [ ]:
pearson_rep_fcrs_long_bw_corr.stats()

### Accuracy: RMSE with nominal waveform (after longterm BW correction)

In [ ]:
rmse_acc_fcrs_long_bw_corr = bps.val.cross_compare(
    fcrs_long_bw_corr,
    [[preproc.nominal_matched]],
    bps.val.rmse_s,
)

In [ ]:
rmse_acc_fcrs_long_bw_corr.boxplot(figsize=(10, 2))

In [ ]:
rmse_acc_fcrs_long_bw_corr.stats()

In [ ]:
rmse_acc_fcrs_long_bw_corr.examples(
    meas_desc, "RMSE with nominal waveform (after longterm BW correction)", good="min"
)

### Accuracy: RMSE with nominal waveform (after full BW correction)

In [ ]:
rmse_acc_fcrs_bw_corr = bps.val.cross_compare(
    fcrs_bw_corr,
    [[preproc.nominal_matched]],
    bps.val.rmse_s,
)

In [ ]:
rmse_acc_fcrs_bw_corr.boxplot(figsize=(10, 2))

In [ ]:
rmse_acc_fcrs_bw_corr.stats()

In [ ]:
rmse_acc_fcrs_bw_corr.examples(
    meas_desc, "RMSE with nominal waveform (after full BW correction)", good="min"
)

### Accuracy: Person correlation with nominal waveform

In [ ]:
person_acc_fcrs_long_bw_corr = bps.val.cross_compare(
    fcrs_long_bw_corr,
    [[preproc.nominal_matched]],
    bps.val.pearson_s,
)

In [ ]:
person_acc_fcrs_long_bw_corr.stats()

In [ ]:
person_acc_fcrs_bw_corr = bps.val.cross_compare(
    fcrs_bw_corr,
    [[preproc.nominal_matched]],
    bps.val.pearson_s,
)

In [ ]:
person_acc_fcrs_bw_corr.stats()

## Cross comparison of cardiac cycles

In [ ]:
ccycles_long_bw_corr = bps.val.build_ccycle_matrix(
    preproc.measured_long_bw_corr, "cc_ref", ccycles_per_fcr
)
ccycles_long_bw_corr.shape

In [ ]:
bps.val.plot_stacked_ccycles(
    preproc.measured_long_bw_corr,
    "cc_ref",
    n_ccycles=ccycles_per_fcr,
    title=f"{meas_desc}; Longterm BW removed",
    overlay=nominal_ccycles,
)

In [ ]:
ccycles_bw_corr_ext = bps.val.build_ccycle_matrix(
    preproc.measured_bw_corr, "cc_ref_ext", ccycles_per_fcr
)
ccycles_bw_corr_ext.shape

In [ ]:
ccycles_bw_corr = bps.val.build_ccycle_matrix(
    preproc.measured_bw_corr, "cc_ref", ccycles_per_fcr
)
ccycles_bw_corr.shape

In [ ]:
bps.val.plot_stacked_ccycles(
    preproc.measured_bw_corr,
    "cc_ref",
    n_ccycles=ccycles_per_fcr,
    title=f"{meas_desc}; BW removed",
    overlay=nominal_ccycles,
)

### Rotation speed

In [ ]:
t_len = pd.DataFrame(
    np.vectorize(lambda s: s.t_onsets[-1] - s.t_onsets[0])(ccycles_bw_corr_ext)
)
t_len.shape

In [ ]:
t_len.boxplot()

### Precision: Cross-RMSE of cardiac cycles after longterm BW removal

In [ ]:
rmse_rep_ccycles_long_bw_corr = bps.val.cross_compare(
    ccycles_long_bw_corr,
    ccycles_long_bw_corr,
    bps.val.rmse_s,
)

In [ ]:
rmse_rep_ccycles_long_bw_corr.stats()

In [ ]:
rmse_rep_ccycles_long_bw_corr.boxplot()

In [ ]:
rmse_rep_ccycles_long_bw_corr.heatmaps(figsize=(15, 8))

In [ ]:
rmse_rep_ccycles_long_bw_corr.examples(
    meas_desc,
    "Cross-RMSE of cardiac cycles after longterm BW removal",
    good="min",
    figsize=(20, 10),
)

### Precision: Cross-Pearson correlation of cardiac cycles after longterm BW removal

In [ ]:
pearson_rep_ccycles_long_bw_corr = bps.val.cross_compare(
    ccycles_long_bw_corr,
    ccycles_long_bw_corr,
    bps.val.pearson_s,
)

In [ ]:
pearson_rep_ccycles_long_bw_corr.stats()

In [ ]:
pearson_rep_ccycles_long_bw_corr.boxplot(r"$\rho$")

### Accuracy: Cardiac cycles after full baseline correction vs. nominal cardiac cycles: RMSE

In [ ]:
rmse_acc_ccycles_bw_corr = bps.val.cross_compare(
    ccycles_bw_corr,
    [nominal_ccycles],
    bps.val.rmse_s,
)

In [ ]:
rmse_acc_ccycles_bw_corr.stats()

In [ ]:
rmse_acc_ccycles_bw_corr.boxplot()

In [ ]:
rmse_acc_ccycles_bw_corr.examples(
    meas_desc,
    "Cardiac cycles after full baseline correction vs. nominal cardiac cycles: RMSE",
    good="min",
    figsize=(20, 10),
)

### Accuracy: Cardiac cycles after full baseline correction vs. nominal cardiac cycles: Pearson

In [ ]:
pearson_acc_ccycles_bw_corr = bps.val.cross_compare(
    ccycles_bw_corr,
    [nominal_ccycles],
    bps.val.pearson_s,
)

In [ ]:
pearson_acc_ccycles_bw_corr.boxplot(r"$\rho$")

In [ ]:
pearson_acc_ccycles_bw_corr.stats()

## Cam evaluation

In [ ]:
fcrs_no_wander_ys = [s.y for s in fcrs_bw_corr.squeeze()]
min_turn_len = min(map(len, fcrs_no_wander_ys))
fcrs_no_wander_ys = np.array([y[:min_turn_len] for y in fcrs_no_wander_ys])
averaged_fcr = np.mean(fcrs_no_wander_ys, axis=0)

In [ ]:
bps.val.plot_cam_vs_measured(preproc.nominal_matched, averaged_fcr, cam_params)

In [ ]:
all_acc_fcrs = pd.concat(
    [
        fcr_comp.data[["fcr_m", 0]].set_index(["fcr_m"])
        for fcr_comp in [
            rmse_acc_fcrs_long_bw_corr,
            person_acc_fcrs_long_bw_corr,
            rmse_acc_fcrs_bw_corr,
            person_acc_fcrs_bw_corr,
        ]
    ],
    axis=1,
)
all_acc_fcrs.columns = [
    "rmse_acc_fcrs_long_bw_corr",
    "person_acc_fcrs_long_bw_corr",
    "rmse_acc_fcrs_bw_corr",
    "person_acc_fcrs_bw_corr",
]

In [ ]:
all_rep_fcrs = pd.concat(
    [
        fcr_comp.data.set_index(["fcr_m", "fcr_r"])
        for fcr_comp in [
            rmse_rep_fcrs_long_bw_corr,
            pearson_rep_fcrs_long_bw_corr,
        ]
    ],
    axis=1,
)
all_rep_fcrs.columns = [
    "rmse_rep_fcrs_long_bw_corr",
    "pearson_rep_fcrs_long_bw_corr",
]

In [ ]:
with h5py.File(
    results_folder / meas_file.with_suffix("").with_suffix(".r.hdf5").name, "w"
) as f:
    preproc.nominal_matched.to_hdf(f.create_group("nominal_matched"))
    preproc.measured_long_bw_corr.to_hdf(f.create_group("measured_long_bw_corr"))
    preproc.measured_bw_corr.to_hdf(f.create_group("measured_bw_corr"))
    cam_params.to_hdf(f.create_group("cam_params"))
    cam_inst_params.to_hdf(f.create_group("cam_inst"))
    sim_params.to_hdf(f.create_group("simulator"))
    meas_params.to_hdf(f.create_group("meas_params"))

    rmse_acc_fcrs_long_bw_corr.stats_to_hdf(f, "rmse_acc_fcrs_long_bw_corr")
    person_acc_fcrs_long_bw_corr.stats_to_hdf(f, "person_acc_fcrs_long_bw_corr")
    rmse_acc_fcrs_bw_corr.stats_to_hdf(f, "rmse_acc_fcrs_bw_corr")
    person_acc_fcrs_bw_corr.stats_to_hdf(f, "person_acc_fcrs_bw_corr")
    rmse_acc_ccycles_bw_corr.stats_to_hdf(f, "rmse_acc_ccycles_bw_corr")
    pearson_acc_ccycles_bw_corr.stats_to_hdf(f, "pearson_acc_ccycles_bw_corr")
    rmse_rep_fcrs_long_bw_corr.stats_to_hdf(f, "rmse_rep_fcrs_long_bw_corr")
    pearson_rep_fcrs_long_bw_corr.stats_to_hdf(f, "perason_rep_fcrs_long_bw_corr")
    rmse_rep_ccycles_long_bw_corr.stats_to_hdf(f, "rmse_rep_ccycles_long_bw_corr")
    pearson_rep_ccycles_long_bw_corr.stats_to_hdf(f, "pearson_rep_ccycles_long_bw_corr")

    ds_acc_fcrs = f.create_dataset(
        "all_acc_fcrs", data=(noidx := all_acc_fcrs.reset_index()).to_numpy()
    )
    ds_acc_fcrs.attrs["columns"] = noidx.columns.to_numpy()

    ds_rep_fcrs = f.create_dataset(
        "all_rep_fcrs", data=(noidx := all_rep_fcrs.reset_index()).to_numpy()
    )
    ds_rep_fcrs.attrs["columns"] = noidx.columns.to_numpy()